In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
)

import torch, copy, os
from module.score import evaluation_f1
from module.load_json import *
from module.maps import *
from module.args import print_torch_info
from module.inference import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 1


# Paths and Modes

In [3]:
EVAL_MODE = False

RESULT_ID = 'uncleaned_v21_run_1_ensemble'

ACD_1_CHECKPOINT = 'training_results/uncleaned_v21_run_2/acd_klue_roberta_base_mlm_fine_tuned_uncleaned_v21_run_2/checkpoint-12500'
ACD_2_CHECKPOINT = 'training_results/uncleaned_v21_run_1/acd_monologg_koelectra_base_v3_discriminator_uncleaned_v21_run_1/checkpoint-12500'
ACD_3_CHECKPOINT = 'training_results/uncleaned_v21_run_4/acd_snunlp_kr_electra_discriminator_uncleaned_v21_run_4/checkpoint-12500'

ASC_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-1608'
ASC_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-1608'
ASC_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-1608'

TEST_DATA_PATH = 'dataset/nikluge-sa-2022-test.jsonl'
EVAL_DATA_PATH = 'dataset/nikluge-sa-2022-dev.jsonl'

In [4]:
if EVAL_MODE == True:
    TEST_DATA_PATH = EVAL_DATA_PATH
test_data = jsonlload(TEST_DATA_PATH)
print('>>>>> >>>>> >>>>> ', TEST_DATA_PATH, ' <<<<< <<<<< <<<<<', '\n', sep='')

>>>>> >>>>> >>>>> dataset/nikluge-sa-2022-test.jsonl <<<<< <<<<< <<<<<



# Load Models and Tokenizers

In [5]:
acd_model_1 = AutoModelForSequenceClassification.from_pretrained(ACD_1_CHECKPOINT)
acd_model_2 = AutoModelForSequenceClassification.from_pretrained(ACD_2_CHECKPOINT)
acd_model_3 = AutoModelForSequenceClassification.from_pretrained(ACD_3_CHECKPOINT)
acd_tokenizer_1 = AutoTokenizer.from_pretrained(ACD_1_CHECKPOINT)
acd_tokenizer_2 = AutoTokenizer.from_pretrained(ACD_2_CHECKPOINT)
acd_tokenizer_3 = AutoTokenizer.from_pretrained(ACD_3_CHECKPOINT)

asc_model_1 = AutoModelForSequenceClassification.from_pretrained(ASC_1_CHECKPOINT)
asc_model_2 = AutoModelForSequenceClassification.from_pretrained(ASC_2_CHECKPOINT)
asc_model_3 = AutoModelForSequenceClassification.from_pretrained(ASC_3_CHECKPOINT)
asc_tokenizer_1 = AutoTokenizer.from_pretrained(ASC_1_CHECKPOINT)
asc_tokenizer_2 = AutoTokenizer.from_pretrained(ASC_2_CHECKPOINT)
asc_tokenizer_3 = AutoTokenizer.from_pretrained(ASC_3_CHECKPOINT)

In [ ]:
# # v1

# def acd_ensemble(form, acd_pair):
#     acd_model_1.to(device)
#     acd_model_2.to(device)
#     acd_model_3.to(device)
#     acd_model_1.eval()
#     acd_model_2.eval()
#     acd_model_3.eval()

#     acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

#     acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

#     acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

#     with torch.no_grad():
#         acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
#         acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
#         acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

#     acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
#     acd_predictions = acd_outputs.argmax(-1)[0]
    
#     return acd_predictions

In [6]:
# v2

def acd_ensemble(form, acd_pair):
    acd_model_1.to(device)
    acd_model_2.to(device)
    acd_model_3.to(device)
    acd_model_1.eval()
    acd_model_2.eval()
    acd_model_3.eval()

    acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

    acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

    acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

    with torch.no_grad():
        acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
        acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
        acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

    acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
    
    acd_predictions_1 = acd_outputs_1.argmax(-1).squeeze()
    acd_predictions_2 = acd_outputs_2.argmax(-1).squeeze()
    acd_predictions_3 = acd_outputs_3.argmax(-1).squeeze()
    
    if 0 in [acd_predictions_1, acd_predictions_2, acd_predictions_3]:
        return 0
    
    return 1

In [7]:
def asc_ensemble(form, asc_pair):
    asc_model_1.to(device)
    asc_model_2.to(device)
    asc_model_3.to(device)
    asc_model_1.eval()
    asc_model_2.eval()
    asc_model_3.eval()

    asc_encoded_1 = asc_tokenizer_1(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_1 = {k:v.to(device) for k,v in asc_encoded_1.items()}

    asc_encoded_2 = asc_tokenizer_2(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_2 = {k:v.to(device) for k,v in asc_encoded_2.items()}

    asc_encoded_3 = asc_tokenizer_3(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_3 = {k:v.to(device) for k,v in asc_encoded_3.items()}

    with torch.no_grad():
        asc_outputs_1 = asc_model_1(**asc_encoded_1)['logits']
        asc_outputs_2 = asc_model_2(**asc_encoded_2)['logits']
        asc_outputs_3 = asc_model_3(**asc_encoded_3)['logits']

    asc_outputs = asc_outputs_1 + asc_outputs_2 + asc_outputs_3
    
    return asc_outputs

# Inference

In [8]:
for entity_property_pair in entity_property_pairs:
    num_of_ep_pairs = len(entity_property_pair)
    RESULT_SAVE_NAME = f'{RESULT_ID}_num_of_ep_pairs_{num_of_ep_pairs}.json'
    
    pred_data = inference_m_ensemble(acd_ensemble, asc_ensemble, copy.deepcopy(test_data), entity_property_pair)
    
    if EVAL_MODE == False:
        save_path = './'

        jsondump(pred_data, os.path.join(save_path, RESULT_SAVE_NAME))
        pred_data = jsonload(os.path.join(save_path, RESULT_SAVE_NAME))
    
    print(RESULT_SAVE_NAME)    
    print(len(test_data), len(pred_data))
    
    if EVAL_MODE == True:
        print('INFERENCE DATA: ', TEST_DATA_PATH)

        print('EVAL_MODE :', EVAL_MODE)

        result = evaluation_f1(test_data, pred_data)
        print(list(result.items())[0])
        print(list(result.items())[1])
    break

['본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질', '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질', '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질']


 79%|███████▉  | 1685/2127 [41:25<11:09,  1.52s/it]

패키지/구성품#가격 found.
corrected as 패키지/ 구성품#가격


 81%|████████  | 1721/2127 [42:19<10:16,  1.52s/it]

패키지/구성품#가격 found.
corrected as 패키지/ 구성품#가격


100%|██████████| 2127/2127 [52:19<00:00,  1.48s/it]

number of failed: 206
바디로션으로 써도되고차갑게해서 팩 처럼 듬뿍 올려도 좋다고 함.

스무스 도우 퍼프가 내장되었다는데 그동안 써본 에어퍼프들과 달라요

항균퍼프라고 하니 애껴써야징ㅋㅋㅋ...

통에 그다지 투자를 안한 느낌​

소중한사람에게 선물하기 좋은 고품격 화장품으로 받자마자 고급스러움이 묻어나는 한방화장품이에요~

이 스킨 로션이 여드름에 좋기 때문에 사춘기아이들에게는 더욱 좋다고 해요

그리고 제품이 흐르지 않도록 마개(?!)가 같이 있기 때문에 마개를 살짝 들어주기만 하면 바로 간편히 사용 할 수 있어 좋아요

처음에 케이스를 보고는 데일리로 바를 수 있는걸까 고민했는데 막상 열어서 발라보니 전혀 부담스럽지 않아요.

연하면 연한대로 레트로스러우면서도 맑은 무드의 레드.. 벨벳틴트추천 두번, 아니 세번 가능하답니다.

오픈해서 사용하려고 뺀 어플리케이터에 1회 충분히 쓸 만큼, 불편하지 않게 ``적당량만`` 묻어나온다는 사실도 두근!

요즘 피까지나고 난리난 제 입술도 예쁘게 해줬어요.

다른 코스메디칼 브랜드에서는 온라인에만 명시하거나 2~3줄로 어느단계에 쓰면 되는지 적어두는데 이렇게 효능이나 함께 쓰면 좋은제품, 특징, 사용법등을 제대로 알려주는 종이가 함께 동봉되어있어서 진짜 신기하더라구요,

좋은데 ㅠ3ㅠ한가지 아쉬운점이 있다면 ~이렇게 겉으로는컬러확인이 불가능하다는 점 ....

라마레따 가을 매트립이 출시된매트 립홀드는넘나 좋게도 ! 저같은 입술 각질녀들이안심하고 사용할 수 있도록립틴트와 함께오일 탑코트가 듀얼로 되어있는 제품이에요

립오일은 브러쉬보단스파츌러에 가깝게 생겨서립오일을 입술위에 덜어서얹어주기 좋게 되어있어요

탑코트를 얹으면 !뭔가 .. 가을겨울 파티룩에 어울릴 것 같은무드있는 매력 뿜뿜 ~

스킨, 로션이 하나로 !

블랙에 가까운 모발이신분들은 브라운이면 뭔가 안어울릴수있으니 자연스러운 VT코스메틱 그레이브라운컬러 좋을듯해요 !

그런데 하나의 몸통에 아이라이너도 마스카라도 붙어있으니 넘나 좋은것 